<a href="https://colab.research.google.com/github/neuroidss/EEG-GAN-audio-video/blob/main/512_tpu_Stylegan2_Custom_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Training StyleGan2-ADA

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
#!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!curl https://sdk.cloud.google.com | bash

Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [ ]:
!gcloud init

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2"):
    %cd "/content/drive/MyDrive/colab-sg2/stylegan2"
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2
    %cd colab-sg2
    !git clone --branch tpu https://github.com/shawwn/stylegan2.git
#    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2
    !mkdir downloads
    !mkdir datasets

/content/drive/MyDrive/colab-sg2/stylegan2


In [ ]:
%cd "/content/drive/MyDrive/colab-sg2/stylegan2"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
#!git fetch origin
#!git checkout origin/main -- train.py

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [ ]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/drive/MyDrive/sq-512.zip"

!unzip {zip_path} -d /content/

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=PvHfzJhWx69E8YJqua6gRb33ndIFH4&prompt=consent&access_type=offline&code_challenge=HNwHHMYgI0qAnqirNGQXAafsmZkvGwsjEjSF2FK2Z4o&code_challenge_method=S256

Enter verification code: 4/1AX4XfWgeXV9BkYidJ4iSMvI5yTJrEW2bw4Q2NlRhJFwXHmfv935Efy8vdKE

You are now logged in as [dmitry.sukhoruchkin@neuroidss.ru].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [ ]:
project_id = 'encoded-phalanx-326615'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
#update this to the path to your image folder
dataset_path = "/content/sq-512"
#give your dataset a name
dataset_name = "sq-512"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}
#!python dataset_tool.py create_from_images gs://train_with_tpu/datasets/{dataset_name} {dataset_path}

## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
  import os
  import tensorflow.compat.v1 as tf
  tf.disable_eager_execution()
  import pprint
  assert 'COLAB_TPU_ADDR' in os.environ, 'Did you forget to switch to TPU?'
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(tpu_address) as sess:
    devices = sess.list_devices()
  pprint.pprint(devices)
  device_is_tpu = [True if 'TPU' in str(x) else False for x in devices]
  assert True in device_is_tpu, 'Did you forget to switch to TPU?'


[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12617358712557040437),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17279839774406701468),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14322696781348295986),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8172743378948765118),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17687202632222170366),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 3145886685791245835),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 17834120664734970590),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 5488739679684604622),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1589696840748255836),
 _DeviceAttributes(/job:tpu_worker/replica

In [ ]:
print(tpu_address)

grpc://10.25.9.194:8470


In [ ]:
project_id = 'encoded-phalanx-326615'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
!gsutil cp -r /content/drive/MyDrive/colab-sg2/stylegan2/datasets/* gs://train_with_tpu/datasets/


In [ ]:
GCP_PROJECT_ID = 'encoded-phalanx-326615'
PROJECT_NUMBER = '0'


In [ ]:
!gcloud services --project $GCP_PROJECT_ID enable ml.googleapis.com cloudbuild.googleapis.com


Operation "operations/acf.p2-174339478235-5b01bb64-5c8f-4b35-861b-4482a490306f" finished successfully.


In [ ]:
from google.colab import auth
auth.authenticate_user()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
!export TPU_NAME={tpu_address}
#!export RESUME_PKL="/content/drive/MyDrive/sg2-ext_aydao-anime-danbooru2019s-512-5268480.pkl"
#!export RESUME_PKL=./results/00019-stylegan2-sq-256-1gpu-config-a/network-snapshot-000100.pkl
#!export RESUME_KIMG=100.4
#!export RESUME_TIME=43
#!export RESUME_TIME=`math 10*3600 + 13*60 + 49` # 10h 13m 49s
#RESUME_TIME=0*3600 + 44*60 + 31
#!export RESUME_TIME={RESUME_TIME} # 10h 13m 49s

#    resume_pkl="./results/00019-stylegan2-sq-256-1gpu-config-a/network-snapshot-000100.pkl"
#    resume_kimg=100.4
#    resume_time=0*3600 + 44*60 + 31
#    resume_pkl="/content/drive/MyDrive/stylegan2-cat-config-a.pkl"
#    resume_kimg=0
#    resume_time=0

#config="config-f" # StyleGAN 2
config="config-a" # StyleGAN 1

data_dir="gs://train_with_tpu/datasets"
dataset="sq-512"
mirror="False"
metrics="none"
#!export NO_GCE_CHECK=true
total_kimg=25000
resume="./results/00039-stylegan2-sq-512-1gpu-config-a"

!python run_training.py --resume "{resume}" --total-kimg "{total_kimg}" --data-dir "{data_dir}" --config="{config}" --dataset="{dataset}" --mirror-augment="{mirror}" --metrics="{metrics}"


Local submit - run_dir: results/00040-stylegan2-sq-512-1gpu-config-a
dnnlib: Running training.training_loop.training_loop() on localhost...


attach_debugger <function attach_debugger at 0x7efe8d568dd0>
print_status <function print_status at 0x7efe8d568e60>
freeze_forever <function freeze_forever at 0x7efe8d569050>
quit <function quit at 0x7efe8d5690e0>
save_and_quit <function save_and_quit at 0x7efe8d569200>
throw_exception <function throw_exception at 0x7efe8d569290>
 Resuming from "./results/00039-stylegan2-sq-512-1gpu-config-a/network-snapshot-000914.pkl", kimg 914
Using TPU grpc://10.41.146.226:8470
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x55dd91684000 @  0x7eff69032001 0x7eff65b1654f 0x7eff65b66b58 0x7eff65b6ab17 0x7eff65c09203 0x55dd8aebb544 0x55dd8aebb240 0x55dd8af2f627 0x55dd8af29ced 0x55dd8aebd48c 0x55dd8aefe159 0x55dd8aefb0a4 0x55dd8aebd698 0x55dd8af2bfe4 0x55dd8af299ee 0x55dd8adfbe2b 0x55dd8af2bfe4 0x55dd8af29ced 0

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.